In [19]:
import gym
import random
from keras import Sequential
import tensorflow as tf
from collections import deque
from keras.layers import Dense
from keras.optimizers import adam
import matplotlib.pyplot as plt
from keras.activations import relu, linear

import numpy as np

In [20]:
env = gym.make('LunarLander-v2')
# Landing pad is always at coordinates (0,0). 
# Coordinates are the first two numbers in state vector. 
# Reward for moving from the top of the screen to landing pad and zero speed is about 100..140 points. 
# If lander moves away from landing pad it loses reward back. 
# Episode finishes if the lander crashes or comes to rest, receiving additional -100 or +100 points. 
# Each leg ground contact is +10. 
# Firing main engine is -0.3 points each frame. Solved is 200 points. 
# Landing outside landing pad is possible. 
# Fuel is infinite, so an agent can learn to fly and then land on its first attempt. 
# Four discrete actions available: do nothing, fire left orientation engine, fire main engine,
#     fire right orientation engine.

print(env.observation_space)
print(env.observation_space.high)
print(env.observation_space.low)
state_size = env.observation_space.shape[0]

Box(8,)
[inf inf inf inf inf inf inf inf]
[-inf -inf -inf -inf -inf -inf -inf -inf]


In [21]:
# # neural network architecture, input=states, output=actions
# def build_model(state_size, action_size):
#     model = Sequential()
#     model.add(Dense(150, input_dim=state_size, activation=relu))
#     model.add(Dense(120, activation=relu))
#     model.add(Dense(action_size, activation=linear))
#     model.compile(loss='mse', optimizer="Adam")
#     return model

def build_model(input_size, output_size):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(128, input_dim = input_size, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(output_size, activation = "linear"))
    model.compile(loss='mse', optimizer="Adam")
    return model

![Explanation of environment exploration/exploitation] (https://github.com/VeronikaRevjakina/LunaLander_RL/blob/master/firefox_mQVLhQrl3o.png?raw=true)

[QL](https://www.freecodecamp.org/news/diving-deeper-into-reinforcement-learning-with-q-learning-c18d0db58efe)

In [46]:
def dql_train(env, state_size, action_size, model, train_games_num, max_steps_in_game,batch_size,discount_rate,
              epsilon=1,epsilon_decay = .99):
    scores = np.array([])
    #object for remembering all things need for dql in memory
    memory = {"state":np.array([]), 
            "action":np.array([], dtype = int), 
            "reward":np.array([]),
            "new_state":np.array([]), 
            "done":np.array([])
            }
    for game in range(train_games_num):
        obs = env.reset() #to initial state
        obs = obs.reshape([1,state_size]) # reshape for neural network input
#         np.reshape(obs, (1, state_size))
#         obs = np.reshape(obs, (1, state_size))
        game_score = 0
        
        for step in range(max_steps_in_game):  
            # get an action number random or model
            if np.random.rand() <= epsilon:
                action =int(random.randrange(action_size))  # get random because explore env
            else:
                act_values = model(tf.convert_to_tensor(obs)) # model predict based on state
                action = int(np.argmax(act_values[0]))
            env.render()
            # take an action
            new_obs, reward, done, info = env.step(action)
            new_obs = new_obs.reshape([1,state_size]) # reshape for neural network input
            game_score += reward
            
            #  remember into memory this step
            if len(memory["state"])>0:
                memory["state"] = np.vstack((memory["state"], obs)) # to get matrix of row = state vector of 8 values
                memory["new_state"] = np.vstack((memory["new_state"], new_obs))
            else:
                memory["state"] = np.array(obs) # if not initialized new
                memory["new_state"] = np.array(new_obs)
            memory["action"] = np.append(memory["action"], action)  # just vectors
            memory["reward"] = np.append(memory["reward"], reward)
            memory["done"] = np.append(memory["done"], done)

            obs = new_obs  # change to new state
            
            # if we get data for batch then update model weights with earned data, clear it and start again:
            if len(memory["state"])>=batch_size:
                # randomly sample batch from memory with 64 size always
                indexes = np.random.randint(len(memory["done"]), size=batch_size)
                batch = {
                    "state": np.squeeze(memory["state"][indexes]), 
                    "action": memory["action"][indexes], 
                    "reward": memory["reward"][indexes], 
                    "new_state": np.squeeze(memory["new_state"][indexes]), 
                    "done": memory["done"][indexes]
                    }
                # Bellman's equation for target only for chosen actions, 
                #(1-done) to discard cumilative gain for last actions
                targets = batch["reward"] + \
                        discount_rate*(np.amax(model.predict_on_batch(batch["new_state"]), axis=1))*(1-batch["done"])
                targets_full = model.predict_on_batch(batch["state"])  # needed to fill counted above target 
                # on performed actions, rest stays as model counted
                targets_full[np.array(range(batch_size)), batch["action"]] = targets # change specific indexes
#                 ind = np.array([i for i in range(batch_size)])
#                 targets_full1 = tf.constant(targets_full[[ind], batch["action"]]) 
#                 targets_full=targets


                # update model weights
                model.fit(batch["state"], targets_full, epochs = 1, verbose = 0)
                
                if epsilon > 0.01:
                      epsilon *= epsilon_decay  # update exploration/exploitation parameter

            if done: # from step performed
                break

        scores = np.append(scores, game_score)
        if (game+1) % 100 == 0:
            print(f'Avg score for the last {100} games: {np.mean(scores[-100:])}')
    print(f"Evaluate in {game+1} iterations")
    return scores, model
            

In [47]:
games_iter = 800
num_iter = 1000
discount_rate = 0.99  # how much depend on long time reward
batch_size = 64  # for sampling from memory, training 
action_size =4 # fixed for environment

model = build_model(state_size, action_size)  #get model of architecture above

traning_score, trained_model = dql_train(env,state_size,action_size, model, games_iter,num_iter,batch_size,discount_rate)

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [24]:
x=list(range(len(training_score)))
plt.plot(x, training_score)
plt.xlabel("Iteration number")
plt.ylabel("Reward")
plt.show()

NameError: name 'training_score' is not defined

In [ ]:
trained_model.save('trained_model.h5')

In [ ]:
from keras.models import load_model
model = load_model('trained_model.h5')

In [9]:
test_games_num = 100
test_scores = []
max_steps_in_game = 1000
for game in range(test_games_num):
    score = 0
    observation = env.reset()  # get initial state
    for step in range(max_steps_in_game):
        env.render()  # show gif
#         print(observation)  # print state vector 8, fixed for environment
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)  # step returns 4 parameters
        score +=reward
        if done:  # game over need reset
#             print("Episode finished after {} timesteps".format(step+1))
            print(score)
            break
    test_scores.append(score)

-76.09117154009367
-131.60169391064753
-408.2377808717626
-182.96031703311473
-359.96201733608706
-121.26160167794612
-216.90136446776316
-501.2584007026191
-74.72028169081048
-127.00197073999618
-109.30880091204284
-178.78587977693718
-115.91864816550228
-197.01982445381321
-98.54451388120903
10.941737404175583
-289.84081703933884
-321.60963417668245
-127.96548625071343
-283.4306951108398
-105.84525848540619
-90.68991666245802
-197.66198856399382
-104.29179822725567
-333.58161480855966
-388.3357336889067
-147.20942529575166
-118.11830075591402
-123.33705015373206
-70.02705600225505
4.838136517284781
-106.53878603350991
-152.88092331188983
-368.18264031134527
-173.95224217521542
-68.71983727579436
-227.9824360103706
-297.1124843577488
-89.53212830988443
-100.624475821276
-75.33534800608732
-71.3494698211212
-232.4138201448486
-79.53578106721253
-84.90792141697231
-180.88894967853898
-52.64260320556362
-221.9220148920389
-93.9058169619801
-93.29161706209143
-229.02902394468742
-341.4376

In [11]:
print("Average reward on test 100 games: ", np.mean(test_scores))

Average reward on test 100 games:  -172.83697185238046


In [12]:
test_scores = np.array(test_scores)
print("Number of games in 100 where reward")
print("less than 0: ", len(test_scores[test_scores<0]))
print("equals 0: ", len(test_scores[test_scores==0]))
print("more than 0: ", len(test_scores[test_scores>0]))
print("equals or more than 100: ", len(test_scores[test_scores>=100]))
print("equals or more than 200: ", len(test_scores[test_scores>=200]))

Number of games in 100 where reward
less than 0:  97
equals 0:  0
more than 0:  3
equals or more than 100:  0
equals or more than 200:  0


31